In [1]:
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
import re

In [4]:
import os

# Conversion to TEI

In [5]:
class TEI:
    """ EcoCor TEI document object"""
    langmap = {'Englisch':'en', 
               'Deutsch':'de',
               'Orig. Franz. / Englisch': 'en'
              }
    teiid = 1
    def __init__(self, title, author, sourcetxtname):
        self.title = title
        print(title)
        self.author = author
        self.empty = False
        self.sourcetxtname = sourcetxtname.replace('.txt','')
        #print(self.sourcetxtname)
        if sourcetxtname != 'NoSourceTxt':
            self.sourcetxtpath = f'ecocorMD/{self.sourcetxtname}.txt'
            self.outputfilename = self.sourcetxtname
            if not os.path.exists(self.sourcetxtpath):
                #print(sourcetxtname)
                #print(self.sourcetxtpath)
                self.empty = True
        else:
            self.empty = True
            self.sourcetxtpath = None
            self.outputfilename = title
        self.outputfilename = self.outputfilename.replace(' ', '')
            
        self.year = None
        self.wikidataid = None
        self.wikidataidauth = None
        self.lang = 'Undefined'
        with open('aux/EcoStub.xml') as ecostub:
            stubsoup = BeautifulSoup(ecostub, 'xml')
        self.tree = stubsoup
        self.len_words = 0
    
    def isolang(self):
        return self.langmap.get(self.lang)
    
    def measure_length(self, text):
        length = len(re.findall(r'\b\w.*?\b', text))
        #print(length)
        return length 
    
    def wikify_tree(self, treetitle, treeauthor):
        treeauthor['ref'] = self.wikidataidauth
        treetitle['ref'] = self.wikidataid
        
    def checkchapterheader(self, paragraph):
        if paragraph.startswith('##'):
            return True
        return False

    def checkpartheader(self, paragraph):
        if paragraph.startswith('#') and not paragraph.startswith('##'):
            return True
        return False
    
    def checkfront(self, paragraph):
        if paragraph.startswith('^'):
            #print('front')
            #print(paragraph)
            return True
        return False
    
    def get_paragraph_id(self, count):
        return f'{self.get_full_id()}_{count*10}'        
        
        
    def add_text(self):
        teitext = self.tree.find('text')
        teitext.clear()
        body = self.tree.new_tag('body')
        toptag = body
        currentdiv = body
        if self.sourcetxtpath == None:
            self.empty = True
            return 'NoText'
        if not os.path.exists(self.sourcetxtpath):
            self.empty = True
            #print(self.sourcetxtpath)
            return 'NoText'
        with open(self.sourcetxtpath) as opensourcefile:
            #text_raw = opensourcefile.read()
            text_to_insert = opensourcefile.readlines()
            text_raw = '\n'.join(text_to_insert)
        #print(text_raw)[:20]
        #print(text_to_insert)[:2]
        self.len_words = self.measure_length(text_raw)
        pcount=1
        for paragraph in text_to_insert:
            if self.checkchapterheader(paragraph):
                div = self.tree.new_tag('div')
                head = self.tree.new_tag('head')
                head.append(paragraph.strip('#'))
                div['type'] = 'chapter'
                div.append(head)
                toptag.append(div)
                currentdiv = div
            elif self.checkpartheader(paragraph):
                div = self.tree.new_tag('div')
                head = self.tree.new_tag('head')
                head.append(paragraph.strip('#'))
                div['type'] = 'group'
                div.append(head)
                body.append(div)
                toptag = div
                currentdiv = div
                #print('Div found')
                #print(self.title)
            elif self.checkfront(paragraph):
                front = self.tree.new_tag('front')
                p = self.tree.new_tag('p')
                p['xml:id'] = self.get_paragraph_id(pcount)
                pcount+=1
                p.append(paragraph.strip('^ \n'))
                if len(paragraph.strip('^')) > 0:
                    front.append(p)
                teitext.append(front)
                currentdiv = front
            else:
                p = self.tree.new_tag('p')
                p['xml:id'] = self.get_paragraph_id(pcount)
                pcount+=1
                p.append(paragraph)
                if len(paragraph.strip()) > 0: 
                    currentdiv.append(p)
        teitext.append(body)
    
    def get_full_id(self):
        numzeros = 5 if TEI.teiid < 10 else 4
        return f'eco_{self.isolang()}_{"0"*numzeros}{TEI.teiid}'
    
    def update_tree(self):
        #def updatetag(tagname, tagcontent):
        #    treetag = self.tree.find(tagname)
            
        treetitle = self.tree.find('title') # find title tag in the stub
        treetitle.clear() # remove text from stub
        treetitle.append(self.title) # add current title
        treeauthor = self.tree.find('author')
        treeauthor.clear() # remove text from stub
        treeauthor.append(self.author) # add current title
        
        
        self.wikify_tree(treetitle, treeauthor) ## add wikidata links
        
        self.add_text()
        #print(self.tree.find('body'))
        
        numpages = self.tree.find('measure')
        numpages.clear()
        numpages.append(str(self.len_words))
        
        root = self.tree.find('TEI')
        root['xml:id'] = self.get_full_id()
        TEI.teiid+=1
        #print(self.isolang())
        root['xml:lang'] = self.isolang()
    
    
    def serialize(self):
        """SERIALISATION"""
        self.update_tree()
        return self.tree.prettify()
    
    def choose_folder(self):
        if self.empty:
            return 'no_text_yet' 
        return 'has_text'
        
    def output_TEI(self):
        with open(f'tei/{self.isolang()}/{self.choose_folder()}/{self.outputfilename}.xml', 
                  'w') as outfile:
            outfile.write(self.serialize())
        
        

In [10]:
def row_to_tei(row):
    author = row['Autor*in']
    title = row['Titel']
    sourcetxtname = row['Filename']
    new_tei = TEI(title, author, sourcetxtname)
    new_tei.year = row['Jahr']
    new_tei.wikidataid = row['Wiki-Data ID Work'].replace('/wiki/','/entity/')
    new_tei.wikidataidauth = f'https://www.wikidata.org/entity/{row["Wiki-Data ID Author"]}'
    new_tei.lang = row['Sprache']
    new_tei.output_TEI()
    return f'{sourcetxtname} success'

In [11]:
df = pd.read_excel('aux/EcoCorMetadata.xlsx')

In [12]:
df['Filename'] = df['Filename'].fillna('NoSourceTxt')

In [14]:
TEI.teiid = 1
df.apply(row_to_tei, axis=1)

Utopia
Nova Atlantis
Robinson Crusoe
The Mysteries of Udolpho
Frankenstein
Moby-Dick
The Last Man
The Paradise of Bachelors and the Tartarus of Maids
Little House on the Prairie
Paris in the Twentieth Century
The Purchase of the North Pole
Tess of the d’Urbervilles 
The American Claimant 
Earth Revisited
The Aerial Brickfield
Dracula
A Corner in Lightning
The Wreck of the South Pole, or the Great Dissembler 
Heart of Darkness
The White Battalions
The Evacuation of England: The Twist in the Gulf Stream
The Great Weather Syndicate 
England's The Air Trust
The Man Who Rocked The Earth
Erewhon
Wuthering Heights
After London, or Wild England
The Hoosier Schoolmaster: A Story of Backwoods Life in Indiana
Deephaven
A Crystal Age
Wilhelm Meisters Lehrjahre
Heinrich von Ofterdingen
Das Erdbeben in Chili
Die Elfen
Das fremde Kind
Die Bergwerke zu Falun
Lebens-Ansichten des Katers Murr
Wilhelm Meisters Wanderjahre
Die Judenbuche
Der Hochwald
Bunte Steine
Der Grüne Heinrich (zweite Fassung)
Nachso

0                                 NoSourceTxt success
1                                 NoSourceTxt success
2                                 NoSourceTxt success
3                      1794_Radcliffe_Udolpho success
4                   1818_Shelley_Frankenstein success
                           ...                       
63            1836_Chamisso_Reise-um-die-Welt success
64     1789_White_Natural-History-of-Selborne success
65                    1850_Cooper_Rural-Hours success
66                        1854_Thoreau_Walden success
67    1911_Muir_My-First-Summer-in-the-Sierra success
Length: 68, dtype: object